In [1]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestNeighbors

In [162]:
blind = pd.read_csv("../Data/Data_submission/test_blind.csv", sep=";") #fichier csv qui sera merge pas les prédiction et soumis sur Kaggle
presences = pd.read_excel("../Data/Data_train/Presences_Absences_train.xlsx") #fichier csv presences/absences

In [192]:
def k_voisins(k, data, test):
    """
    données:
        k: nombre de voisins considérés 
        data: dataFrame contenant entre autre les patch_ID et les espèces associées du train-PA
        test: dataFrame provenant du CSV Kaggle contenant les informations sur les lieux où la prédiction
        doit être faite
    fonction: 
        Liste les espèces présentes dans les k-voisins les plus proches de chaque coordonnées du dataFrame
        test à partir des information de data. 
    résultat:
        resultat: Un fichier CSV contenant les prédicitions d'especes dans la colonne Predicted 
    """
    # data frame contenant les espèces_id selon la lat,lon où l'espèce est recensées (sans doublons)
    data_sans_doublons = data.groupby(['lat', 'lon'])['speciesId'].apply(set).reset_index()
    
    # Données utilisées par l'algo 
    X1 = test[['lat', 'lon']].values
    X2 = data_sans_doublons[['lat', 'lon']].values
    
    # Création du modèle
    nn = NearestNeighbors(n_neighbors=k)
    nn.fit(X2)
    # le modèle retourne l'indice des k-voisins ainsi que leurs distances du point
    distances, indices = nn.kneighbors(X1)
    
    # Récupération des espèces selon les indices des coordonnées des voisins
    result = []
    for i in range(len(test)):
        nearest_species = set()
        for j in range(len(indices[i])):
            index = indices[i][j]
            nearest_species.update(data_sans_doublons.loc[index]['speciesId'])
        result.append(list(nearest_species))
    
    #traitement pour l'export du CSV au format acceptable par Kaggle
    test['nearest_species'] = result
    test = test[['Id', 'nearest_species']]
    test = test.rename(columns={'nearest_species': 'Predicted'})    
    test['Predicted'] = test['Predicted'].apply(lambda x: str(x).replace('[', '').replace(']', '').replace(',', ''))
    test.to_csv('4-voisins.csv', index=False, sep=',')

    return test


In [191]:
%%time
test = k_voisins(4, presences, blind)

Wall time: 13.7 s
